# 训练介绍

ModelScope提供了很多模型，大部分模型可以直接在[推理中使用](./模型的推理Pipeline.ipynb)，但是有一些模型被加载到代码中后，有一小部分的模型参数需要根据数据集进行重新定制，这个过程叫做finetune（微调）。

举例来说，用户在modelhub中搜索，会发现有一些模型名称中包含"backbone"字样。如果把这个模型加载到一个分类任务中，这个分类任务模型最后的一部分参数（一般我们把这部分叫分类器）是随机初始化的，这种随机参数不能用来直接进行预测，而要先
经过用户提供的数据集进行训练，生成的模型才能用来预测和使用。

那么为什么不提供完整的模型直接使用呢？这涉及到人工智能领域的一个问题，也就是"泛化性"。比如ModelScope官方提供了一个对通用医疗图片进行分类的模型，这个模型在通常的医疗场景下可以很好的运行，但假如用户的场景是医疗下面的细分领域，
比如牙医，那么牙医特有的图片在这个通用模型下分类效果可能会不好，这就是泛化性问题。泛化性问题在人工智能领域是必然存在的，任何模型都会存在这个问题。于是就会有类似这样的解决方案：首先使用医疗领域的大量数据集训练一个
通用的模型，但这个模型缺少最上面的分类器。用户只需要提供牙医的少量数据，就可以在这个通用模型基础上将分类器训练（微调）出来，达到特定场景适配的要求，这个整体方案就是预训练，提供的通用模型就是backbone。当然，预训练的通用模型训练是无标注的，微调过程的训练是
有标注的，关于这些知识用户可以自行查阅。

在非预训练的解决方案中，比如LSTM、RNN等规模较小的模型，它的参数是接近随机初始化的，这样的模型要经过特定数据集的训练才能够使用。

一般来说，训练过程包含了训练(train)和评估(evaluate)两个过程。
用户可以理解为训练过程是更新模型参数，评估过程穿插在训练过程中或在训练过程之后，给出当前模型好坏的日志，或通过评估结果存储效果最好的模型便于后续训练或使用。
和训练过程穿插进行的评估过程叫做交叉验证。

ModelScope提供了完整的训练组件，其中的主要组件被称为trainer（训练器），这些组件可以在`预训练`或`普通训练`场景下使用。

# 一个简单的例子：文本分类
下面以一个简单的文本分类任务为例，演示如何通过十几行代码，就可以端到端执行一个finetune任务。整体流程包含一下步骤：

- 载入数据集
- 数据预处理
- 训练
- 评估
## 载入数据集
ModelScope可以提供了标准的`MsDataset`接口供用户进行基于ModelScope生态的数据源加载。下面以加载NLP领域的afqmc（Ant Financial Question Matching Corpus）数据集为例进行演示


In [1]:
from modelscope.msdatasets import MsDataset
# 载入训练数据，注意这个训练数据集不是完整的，而是一个小规模测试用的数据集
train_dataset = MsDataset.load('clue',  subset_name='afqmc', split='train')
# 载入评估数据
eval_dataset = MsDataset.load('clue',  subset_name='afqmc', split='validation')



具体MsDataset 使用可以参考接口文档： [数据的处理](../数据集/数据集介绍.ipynb)
## 数据预处理
在ModelScope中，数据预处理与模型强相关，因此，在指定模型以后，ModelScope框架会自动从对应的模型的模型卡片(modelcard)中读取配置文件中的preprocessor关键字，自动完成预处理的实例化。

预处理具体介绍可以参考 [数据预处理器](./数据的预处理.ipynb)。



In [1]:
# 指定文本分类模型，预处理过程trainer会自动完成
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-tiny'


配置文件的相关字段：


In [1]:
...
"preprocessor":{
    "type": "sen-cls-tokenizer",
  },
...


当然，对于进阶的使用者，配置文件也支持用户自定义并从任意本地路径读取，具体参考文档：[configuration详解](../开发者使用指南/Configuration详解.ipynb)
## 训练
首先，配置训练所需参数：


In [1]:
from modelscope.trainers import build_trainer

# 指定工作目录
tmp_dir = "/tmp"

# 配置参数
kwargs = dict(
        model=model_id,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        work_dir=tmp_dir)



根据参数实例化trainer对象


In [1]:
trainer = build_trainer(default_args=kwargs)



最后，调用train接口进行训练


In [1]:
trainer.train()



恭喜，你完成了一次模型训练😀

需要注意的是，上例中`nlp_structbert_sentence-similarity_chinese-tiny`模型是一个已经训练好的模型，因此不必再进行训练可直接使用。
这类NLP模型中一般带有label_mapping.json文件表示label和id映射关系用于推理，用户如果训练它可能会报错label不匹配。
这时需要更新数据集信息，请参考<a href="#load_dataset">载入数据集</a>的章节。

## 评估
训练完成以后，配置评估数据集，直接调用trainer对象的evaluate函数，即可完成模型的评估，


In [1]:
# 直接调用trainer.evaluate，可以传入train阶段生成的ckpt
# 也可以不传入参数，直接验证model
metrics = trainer.evaluate(checkpoint_path=None)
print(metrics)



## 当前版本ModelScope支持finetune的任务和模型列表
以下模型详细调用信息请参考平台官网[https://www.modelscope.cn/](https://www.modelscope.cn/#/models)，对应模型的模型卡片（ModelCard）。支持finetune和训练的模型，在其模型简介页会有“**支持训练**”的标注。

| 领域 | 任务类型      | 模型         |
| --- |-----------|------------|
| 自然语言处理 | 文本分类      | structbert |
|  | 文本分类（多语言） | veco       |
|  | 序列标注      | structbert |
|  | 文本生成      | palm_v2    |
|  | 文本生成      | plug       |
|  | 文本分类      | bert       |
|  | 序列标注      | bert       |
|  | 通用翻译      | csanmt     |
| 图像 | 图像实例分割    | swin-b     |
|  | 图像去噪声     | nafnet     |
|  | 图像颜色增强    | csrnet     |
|  | 人像增强      | gpen       |

# 训练组件的详细介绍

## 更新读入配置的参数

很多时候配置文件的参数无法满足您的需求，或某些配置参数一定要在运行时填入，比如数据集信息、预处理器传入的数据集的key、optimizer的lambda方法、lr_scheduler的迭代次数等，这时就需要在运行时动态调整参数。
下面的代码演示了默认PyTorch trainer中更新参数的一般方式：


In [1]:
import os
from modelscope.utils.hub import read_config
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
train_dataset = MsDataset.load('clue',  subset_name='afqmc', split='train')
eval_dataset = MsDataset.load('clue',  subset_name='afqmc', split='validation')
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
# 读取model中的cfg文件
cfg = read_config(model_id)
# 直接更新其中的参数
cfg.train.max_epochs = 5
cfg.preprocessor.train['label2id'] = {'0': 0, '1': 1}
cfg.preprocessor.val['label2id'] = {'0': 0, '1': 1}
cfg.train.work_dir = '/tmp'
cfg_file = os.path.join('/tmp', 'config.json')
cfg.dump(cfg_file)
kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    cfg_file=cfg_file)
trainer = build_trainer(default_args=kwargs)
trainer.train()



## <div id='load_dataset'>载入数据集</div>

ModelScope的训练过程支持从外部传入数据集实例，也支持将数据集配置写入配置文件中由trainer自动读取。有关数据集在数据文件中的配置可以参考[数据集的配置文件](../开发者使用指南/Configuration详解.ipynb)。

一般而言，ModelScope模型的配置中不会包含具体的数据集信息，如果您需要在配置文件中配置数据集信息（而不是直接外部加载好传入trainer），下面展示了一种基本的做法：


In [1]:
import os
from modelscope.utils.hub import read_config
from modelscope.trainers import build_trainer
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
# 读取model中的cfg文件
cfg = read_config(model_id)
# 直接更新其中的参数
cfg.train.max_epochs = 5
cfg.train.work_dir = '/tmp'
# 为preprocessor设置句子1，句子2，标签的key
cfg.preprocessor.train['label2id'] = {'0': 0, '1': 1}
cfg.preprocessor.val['label2id'] = {'0': 0, '1': 1}
cfg.preprocessor.label='label'
# 设置训练数据集和验证数据集
cfg.dataset = {
    'train': {
        'name': 'clue',
        'subset_name': 'afqmc',
        'split': 'train',
    },
    'val': {
        'name': 'clue',
        'subset_name': 'afqmc',
        'split': 'validation',
    },
}
# 另存为新的配置
cfg_file = os.path.join('/tmp', 'config.json')
cfg.dump(cfg_file)
# 传入cfg_file，传入后cfg_file会替代model_dir中的cfg_file
kwargs = dict(
    model=model_id,
    cfg_file=cfg_file)
trainer = build_trainer(default_args=kwargs)
trainer.train()



trainer可以支持输入数据集是一个列表，在这时，trainer内部会使用[TaskDataset](#TaskDataset)对数据集进行shuffle来训练。

## 预处理器

数据预处理过程可以由您在训练之前手动调用，也可以由trainer在执行数据训练/评估时自动调用。trainer自动调用预处理过程的伪代码类似于：


In [1]:
for i, mini_batch in enumerate(data_loader):
    mini_batch = preprocesser(mini_batch)
    loss = model.forward(mini_batch)
    ...



以下场景建议您训练之前手动调用预处理过程：
- 数据量过大、数据处理比较复杂导致训练时预处理过程较大影响了运行时长
- 数据预处理的临时结果需要被缓存下来以备下次使用
- 预处理器在配置文件中不好构造出来

除此之外的场景您可以使用自动预处理，以简化自定义代码的复杂度。

训练之前手动调用预处理过程的代码可以参考：


In [1]:
import os
from modelscope.utils.hub import read_config, snapshot_download
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.preprocessors import Preprocessor
from transformers import default_data_collator
# MsDataset对于map的支持来自于huggingface的datasets
train_dataset = MsDataset.load('clue', subset_name='afqmc', split='train').to_hf_dataset()
eval_dataset = MsDataset.load('clue', subset_name='afqmc', split='validation').to_hf_dataset()
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
model_dir = snapshot_download(model_id)
# 读取model中的cfg文件
cfg = read_config(model_dir)
cfg.train.work_dir = '/tmp'
# config的预处理字段置为空
cfg.preprocessor = None
cfg_file = os.path.join('/tmp', 'config.json')
cfg.dump(cfg_file)

train_preprocessor = Preprocessor.from_pretrained(model_dir,
                                                  preprocessor_mode='train',
                                                  first_sequence='sentence1',
                                                  second_sequence='sentence2',
                                                  label='label',
                                                  label2id={'0':0, '1':1},
                                                  sequence_length=256)
eval_preprocessor = Preprocessor.from_pretrained(model_dir,
                                                 preprocessor_mode='eval',
                                                 first_sequence='sentence1',
                                                 second_sequence='sentence2',
                                                 label='label',
                                                 label2id={'0':0, '1':1},
                                                 sequence_length=256)
train_dataset = train_dataset.map(train_preprocessor)
eval_dataset = eval_dataset.map(eval_preprocessor)
# 传入cfg_file，传入后cfg_file会替代model_dir中的cfg_file
kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # trainer内部使用了torch.utils.data.dataloader.default_collate, 这个collator和hf的list返回格式不兼容，需要使用hf自己的data_collator
    data_collator=default_data_collator,
    cfg_file=cfg_file)
trainer = build_trainer(default_args=kwargs)
trainer.train()



总结来说，预处理在训练之前定义要注意几个地方：
1. 如果使用MsDataset进行map处理需要调用to_hf_dataset
2. cfg中的preprocessor需要置为空值
3. data_collator需要调整

如果加载数据集的过程是在配置文件中由trainer自动完成，那么trainer也会检查预处理器在文件中的配置，并自动在训练过程中使用，这时请注意更新preprocessor部分的值：


In [1]:
# preprocessor_mode不需要传入，其他的值根据特定模态选择填入
cfg.preprocessor.first_sequence='sentence1'
cfg.preprocessor.second_sequence='sentence2'
cfg.preprocessor.label='label'
cfg.preprocessor.label2id={'0':0, '1':1}




### 如何实现一个预处理器

如果您需要实现一个自己的预处理器用于训练或推理，请查看[这里](./数据的预处理.ipynb)。

## TaskDataset

TaskDataset是ModelScope特有的设计。和模型、预处理器一样，这个组件也是通过注册进行自动调用的。一般来说，模型的训练过程是加载数据集、预处理、训练评估三部分，
但在一些特殊情况下，数据集需要在特定任务或特定模型下又特殊的处理，举例来说：
1. 某个模型在训练过程中需要同时加载多个训练数据集，以某种方式进行shuffle再输入dataloader
2. 某个任务需要加载一个正例和若干个负例，并将正负例在输入时进入一个batch

TaskDataset就是为了这样的场景而设计，也就是说，满足某些任务和模型的特定情境下需要（多个）数据集整体视角下的处理。
TaskDataset的基类如下：


In [1]:
class TaskDataset(ABC):
    """The task dataset base class for all the task specific dataset processors.
    """

    def __init__(self,
                 datasets: Union[Any, List[Any]],
                 mode,
                 preprocessor=None,
                 **kwargs):
        super().__init__()
        self.mode = mode
        self.preprocessor = preprocessor
        self._inner_dataset = self.prepare_dataset(datasets)

    @abstractmethod
    def prepare_dataset(self, datasets: Union[Any, List[Any]]) -> Any:
        """Prepare a dataset.

        User can process the input datasets in a whole dataset perspective.
        This method also helps to merge several datasets to one.

        Args:
            datasets: The original dataset(s)

        Returns: A single dataset, which may be created after merging.

        """
        pass

    @abstractmethod
    def prepare_sample(self, data):
        """Preprocess the data fetched from the inner_dataset.

        If the preprocessor is None, the original data will be returned, else the preprocessor will be called.
        User can override this method to implement custom logics.

        Args:
            data: The data fetched from the dataset.

        Returns: The processed data.

        """
        pass


可以看到，TaskDataset默认关注mode（训练还是评估），是否有预处理过程，以及实际要操作的数据集。
trainer可以接受一个TaskDataset类型的输入（取代原本的数据集输入），但我们一般建议由trainer在内部自动调起TaskDataset，用户只需要传入自己的数据集对象即可。
在这时，mode和preprocessor（如trainer中存在）会由trainer自动传入，_inner_dataset是您在构造中或配置中定义的训练/评估数据集，也会被trainer传入init中。

TaskDataset的注册是任务+模型式的，也就是说，只要在模型文件中指定了模型名称和任务模型，在训练时对应的TaskDataset就会被使用。

### 写一个新的TaskDataset
如果您使用的是Pytorch，您可以直接继承TorchTaskDataset，它继承了PyTorch的Dataset：


In [1]:
from modelscope.msdatasets.task_datasets import TorchTaskDataset
from typing import Any, List, Union
class CustomDataset(TorchTaskDataset):

    def __init__(self,
                 datasets: Union[Any, List[Any]],
                 mode,
                 preprocessor=None,
                 **kwargs):
        super().__init__(datasets, mode, preprocessor, **kwargs)

    def __getitem__(self, item):
        # TODO Write how to fetch an data item here
        pass

    def prepare_dataset(self, datasets: Union[Any, List[Any]]) -> Any:
        # TODO This will be called when the trainer is initing, so you can 
        # write how to mix or prepare the input datasets.
        pass
    def prepare_sample(self, data):
        # how to prepare the sample, by default it will call the preprocessor(if exists in trainer) to do this.
        pass



在您的main方法或其他会被执行的地方调用：


In [1]:
from modelscope.msdatasets.task_datasets import TASK_DATASETS
TASK_DATASETS.register_module(module_name='my-custom-model', group_key='my-custom-task', module_cls=CustomDataset)



这样，在trainer使用您的模型进行训练时就会自动加载这个TaskDataset了。值得注意的是，TaskDataset init结束后会被传入trainer对象，您可以
在您的TaskDataset内部使用它，比如获得配置：


In [1]:
print(self.trainer.cfg)



## 模型的验证
很多模型的配置中，训练过程没有配置交叉验证。如果您想在训练时同步进行交叉验证可以在configuration.json中添加一个EvaluationHook，具体配置如下：
```json
{
   ...
  "train": {
     ...
      "hooks": [
          ...
        {
          "type": "EvaluationHook",
          # 是否每个epoch进行一次验证，false的时候代表按iter验证
          "by_epoch": false,
          # 多少个epoch/iter进行一次验证
          "interval": 100
      }]
  },

}
```

用户可以根据自己实际情况进行调整配置文件，也可自行注册相应hook，并通过type字段注册在配置文件中进行调用。
关于hook的详细说明请参考文档：[回调函数机制详解](../开发者使用指南/回调函数机制详解.ipynb)


## 模型的保存与后续使用

模型训练结束后（或抛异常后），都会有后续的其他处理，比如训练过程抛异常后您可能需要加载最后一个保存下来的模型进行继续训练，比如训练完成后您可能
需要将其用于推理中或导出为中间格式便于在C++环境中使用或进行加速。ModelScope的trainer支持您进行上述的任何一种实操。

### 模型保存的配置

ModelScope支持两种模型保存的Hook，分别是CheckpointHook和BestCkptSaverHook。 

#### CheckpointHook

这是ModelScope`默认`使用的存储模型hook，如果您在配置文件中没有配置，它会被默认添加到hooks中，它的作用是每个epoch存储一次模型。如果您想要覆盖这一效果，
您可以这样配置：
```json
{
   ...
  "train": {
     ...
      "hooks": [
          ...
        {
            # Save every 100 iters(not epochs)
            "type": "CheckpointHook",
            "by_epoch": false,
            "interval": 100
        }]
  },

}
```

#### BestCkptSaverHook

一般来说用户可能会希望只存储最佳的模型文件，ModelScope也提供了对应的机制来达到这一目的。
- 首先需要保证eval_dataset会被传入trainer（或通过配置文件的方式）
- 配置文件增加或改为BestCkptSaverHook

```json
{
   ...
  "train": {
     ...
      "hooks": [
          ...
        {
            "type": "BestCkptSaverHook",
            "metric_key": "accuracy",
            "by_epoch": false,
            "rule": "max",
            "interval": 100
        }]
  }
}
```

"metric_key"用来表示Metric返回值中用来实际进行比较的key，以"rule"的方式比较并存储模型文件。有关Metric的具体使用可以参考[这里](./模型的评估.ipynb)。

#### 使用文件进行训练或推理

以上任意一种CkptHook都会存储两种类型的文件。

- 用来恢复训练的文件
在对应hook的save_dir（如果不指定，即为trainer.work_dir）中，存在后缀名为pth的文件，这个文件是用来恢复训练的。文件里面存储了模型参数、optimizer参数、
lr_scheduler参数、随机状态、trainer状态等，加载后就可以继续训练（包括dataloader的后续读取数据）。加载方式可以参考：


In [1]:
import os
trainer.train(os.path.join(trainer.work_dir, 'a-pth-file.pth'))




- 用来推理的文件
在对应的save_dir中会存在一个output文件夹，打开后会发现里面的文件格式和推理使用的文件相同，您可以在pipeline中直接加载这个文件夹进行推理。
有关pipeline的使用参考[这里](./模型的推理Pipeline.ipynb)。

#### 使用其他框架的dataset进行训练

ModelScope支持使用PyTorch框架的Dataset或其他自定义的Dataset进行训练，只需要该Dataset具有__getitem__方法（或者说，支持可以传入torch的DataLoader中即可）。
您只需要将该dataset传入trainer构造方法的train_dataset和eval_dataset参数中。

#### 使用torch的Module进行训练

ModelScope的trainer支持训练torch.nn.Module，前提是您必须传入对应的configuration.json。

## 数据并行

More information needed。

## 使用deepspeed等分布式框架训练大模型

More information needed。

## 各模态的trainer

### NLP

在NLP场景下，ModelScope提供了领域友好的特定trainer，以下是一个具体的sample：



In [1]:
from modelscope.metainfo import Preprocessors
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.utils.constant import Tasks


# 通过这个方法修改config
def cfg_modify_fn(cfg):
    cfg.task = Tasks.sentence_similarity
    cfg['preprocessor'] = {'type': Preprocessors.sen_sim_tokenizer}
    cfg.train.optimizer.lr = 2e-5
    cfg['dataset'] = {
        'train': {
            # 实际label字段内容枚举，在训练backbone时需要传入
            'labels': ['0', '1'],
            # 第一个字段的key
            'first_sequence': 'sentence1',
            # 第二个字段的key
            'second_sequence': 'sentence2',
            # label的key
            'label': 'label',
        }
    }
    cfg.train.max_epochs = 5
    cfg.train.lr_scheduler = {
        'type': 'LinearLR',
        'start_factor': 1.0,
        'end_factor': 0.0,
        'total_iters':
            int(len(dataset['train']) / 32) * cfg.train.max_epochs,
        'options': {
            'by_epoch': False
        }
    }
    cfg.train.hooks = [{
        'type': 'CheckpointHook',
        'interval': 1
    }, {
        'type': 'TextLoggerHook',
        'interval': 1
    }, {
        'type': 'IterTimerHook'
    }, {
        'type': 'EvaluationHook',
        'by_epoch': False,
        'interval': 100
    }]
    return cfg


work_dir = 'tmp'
model_id = 'damo/nlp_structbert_backbone_base_std'
dataset = MsDataset.load('clue', subset_name='afqmc')
kwargs = dict(
    model=model_id,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    work_dir=work_dir,
    seed=42,
    cfg_modify_fn=cfg_modify_fn)

# 使用nlp-base-trainer
trainer = build_trainer(name='nlp-base-trainer', default_args=kwargs)
# 评估原始模型
origin_result = trainer.evaluate()
# finetune
trainer.train()
# 评估finetune后的模型
finetuned_result = trainer.evaluate()
print(f"原始模型的评估结果为{origin_result}, finetune后模型达到{finetuned_result}")

# 直接运行本示例得到输出：
# 原始模型的评估结果为{'accuracy': 0.31000927090644836}, finetune后模型达到{'accuracy': 0.7509267926216125}



nlp-base-trainer继承了默认的trainer，对于nlp域内多提供了以下功能：

- 提供了cfg_modify_fn参数，用户可以在调用trainer之前在代码中修改cfg，并且基于cfg_modify_fn提供了`NlpTrainerArguments`类将参数扁平化传入
- 在init时会尝试读取configuration.json/config.json/label_mapping.json，将label数量信息（num_labels）传递给模型，将label2id传递给preprocessor
- 支持在cfg的dataset字段中配置first_sequence、second_sequence、label三个字段的信息，以免污染preprocessor配置，支持在dataset字段中配置labels，该labels信息会被解析成label2id和num_labels

`NlpTrainerArguments`是一个带有__call__方法的dataclass类，是参数配置方法（cfg_modify_fn）的高级封装。
它像dataclass一样接受参数，并可以作为callback传递给cfg_modify_fn来修改cfg。我们推荐您优先使用它。


In [1]:
from modelscope.metainfo import Preprocessors
from modelscope.metainfo import Trainers
from modelscope.msdatasets import MsDataset
from modelscope.trainers import NlpTrainerArguments
from modelscope.trainers import build_trainer
from modelscope.utils.constant import Tasks

dataset = MsDataset.load('clue', subset_name='tnews')
train_dataset = dataset['train']
validation_dataset = dataset['validation']
cfg_modify_fn = NlpTrainerArguments(
    # task信息
    task=Tasks.text_classification,
    # 预处理器类型
    preprocessor_type=Preprocessors.sen_cls_tokenizer,
    # 训练数据集的句子1
    train_first_sequence='sentence',
    # 训练数据集的label
    train_label='label',
    # labels列表
    labels=[
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                '11', '12', '13', '14'
            ],
    # 最大迭代轮数
    max_epochs=5,
    # 优化器参数，输入是一个dict。可以只传入关心的值，常见的如lr
    optimizer_args={
        'lr': 3e-5,
    },
    # lr_scheduler参数，输入是一个dict。可以只传入关心的值，如默认配置中的LinearLR的total_iters参数
    lr_scheduler_args={
        'total_iters': int(len(train_dataset) / 32) * 5,
    },
    # 存储Ckpt的Hook
    checkpoint_saving_type='BestCkptSaverHook',
    # BestCkptSaverHook判断是否是最佳模型时使用的Metric key
    metric_key='accuracy',
    train_batch_size_per_gpu=32,
    checkpoint_interval=1,
    train_workers_per_gpu=0,
    checkpoint_by_epoch=False,
    evaluation_interval=1,
    evaluation_by_epoch=False,
    eval_workers_per_gpu=0,
    metrics=['seq-cls-metric'],
)

kwargs = dict(
    model='damo/nlp_structbert_backbone_base_std',
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    work_dir='/tmp',
    seed=42,
    cfg_modify_fn=cfg_modify_fn)

trainer = build_trainer(name=Trainers.nlp_base_trainer, default_args=kwargs)

# 评估原始模型
origin_result = trainer.evaluate()
# finetune
trainer.train()
# 评估finetune后的模型
finetuned_result = trainer.evaluate()
print(f"原始模型的评估结果为{origin_result}, finetune后模型达到{finetuned_result}")



需要注意的是，这个类的所有值默认是None，这时代表使用configuration.json中的默认设置。
有关NlpTrainerArguments的具体设置您可以查看`modelscope.trainers.nlp_trainer.py`。

nlp-base-trainer的其他方法和构造方法参数和默认trainer相同，您可以按照默认trainer的方式使用nlp-base-trainer。
如果您使用nlp的模型并且nlp-base-trainer满足您的需求，我们推荐您在域内优先使用它。
